In [1]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw

def loadMesh(filename: str) -> list[float]:
    v = []
    vt = []
    vn = []

    vertices = []
    normal=[]
    texture=[]
    with open(filename, "r") as file:

        line = file.readline()

        while line:

            words = line.split(" ")
            if words[0]=="v":
                v.append(read_vertex_data(words))
            elif words[0]=="vt":
                vt.append(read_texture_data(words))
            elif words[0]=="vn":
                vn.append(read_normal_data(words))
            
            elif words[0]=="f":
                face = []
                for word in words[1:]:
                    w = word.split('/')

                    face.append(int(w[0]))
                    vertices.append(v[int(w[0])-1])
                    texture.append(vt[int(w[1])-1])
                    normal.append(vn[int(w[2])-1])


                # read_face_data(words, v, vertices)
            
            line = file.readline()

    return vertices,texture,normal
 


def read_vertex_data(words: list[str]) -> list[float]:
    #вершины
    return [
        float(words[1]),
        float(words[2]),
        float(words[3])
    ]
def read_texture_data(words: list[str]) -> list[float]:
    #вершины
    return [
        float(words[2]),
        float(words[3]),
        float(words[4]),
    ]
def read_normal_data(words: list[str]) -> list[float]:
    #вершины
    return [
        float(words[2]),
        float(words[3]),
        float(words[4])
    ]

    
def odnor(coord):
    new=[]
    for i in range(len(coord)):
        new.append(list(coord[i]))
        new[i].append(1)
    return new
img = Image.new("RGB",(2400,2400), (255,255,255))
points=(loadMesh('african_head.obj'))[0]
draw = ImageDraw.Draw(img)
new_c= points
for i in range(0,int(len(new_c))-2,3):
   
    draw.polygon([(600*(2-new_c[i][0]),600*(2-new_c[i][1])),(600*(2-new_c[i+1][0]),600*(2-new_c[i+1][1])),(600*(2-new_c[i+2][0]),600*(2-new_c[i+2][1]))], outline='black')
    
    
img.show()

Роберстсон

In [5]:
from numpy.linalg import solve
def robertson(coord,norm,cam):
    back=[]
    coord=odnor(coord)
    norm=odnor(norm)
    # print(coord)
    
    
    for i in range(0,len(norm)-2,3):
        nx=(norm[i][0]+norm[i+1][0]+norm[i+2][0]+1)/4
        ny=(norm[i][1]+norm[i+1][1]+norm[i+2][1]+1)/4
        nz=(norm[i][2]+norm[i+1][2]+norm[i+2][2]+1)/4
        n=[nx,ny,nz,1]
        new_t=np.dot(cam,n)
        if new_t>0:
            
        # if np.dot(cam,c)>=0:
            back.append(coord[i][0:-1])
            back.append(coord[i+1][0:-1])
            back.append(coord[i+2][0:-1])
        # print(len(back))
    return back

img = Image.new("RGB",(2400,2400), (255,255,255))
points=(loadMesh('african_head.obj'))[0]
draw = ImageDraw.Draw(img)
norm=(loadMesh('african_head.obj'))[2]
new_c= robertson(points,norm,[0,0,1,0])
for i in range(0,int(len(new_c)-2),3):  
     draw.polygon([(600*(2-new_c[i][0]),600*(2-new_c[i][1])),(600*(2-new_c[i+1][0]),600*(2-new_c[i+1][1])),(600*(2-new_c[i+2][0]),600*(2-new_c[i+2][1]))], outline='black')
img.show()     

Z-буффер

In [4]:
from PIL import Image, ImageDraw
import numpy as np

def z_buffer(points,texture,normal, width, height):

    z_buffer = np.full((height, width), np.inf)
    img = Image.new('RGB', (width, height), 'black')
    draw = ImageDraw.Draw(img)
    
    for i in range(0,int(len(points)-2),3):
        triangle=[points[i],points[i+1],points[i+2]]
        nx=(normal[i][0]+normal[i+1][0]+normal[i+2][0]+1)/4
        ny=(normal[i][1]+normal[i+1][1]+normal[i+2][1]+1)/4
        nz=(normal[i][2]+normal[i+1][2]+normal[i+2][2]+1)/4
        n=[nx,ny,nz]

        draw_triangle(triangle,n, draw, z_buffer, width, height)
    
    return img

def draw_triangle(triangle,texture, draw, z_buffer, width, height):

    v0=[0,0,0]
    v1=[0,0,0]
    v2=[0,0,0]

    for i in range(3):
        v0[i], v1[i], v2[i] = 700*(1-triangle[0][i]),700*(1-triangle[1][i]),700*(1-triangle[2][i])
  
    min_x = max(0, min(v0[0], v1[0], v2[0]))
    max_x = min(width - 1, max(v0[0], v1[0], v2[0]))
    min_y = max(0, min(v0[1], v1[1], v2[1]))
    max_y = min(height - 1, max(v0[1], v1[1], v2[1]))
    camera_v = [0, 0,1]
    intensity = np.dot(camera_v, texture)

    if intensity < 0:
        return
    color = int(intensity * 255)
  
    for x in range(int(min_x), int(max_x) + 1):
        for y in range(int(min_y), int(max_y) + 1):
            if is_inside_triangle(x, y, v0, v1, v2):
                z = get_z_value(x, y, v0, v1, v2)
                if z < z_buffer[y, x]:
                    z_buffer[y, x] = z
                    draw.point((x, y), fill=color)

def is_inside_triangle(px, py, v0, v1, v2):

    def sign(p1, p2, p3):
        return (p1[0] - p3[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p3[1])

    b1 = sign((px, py), v0, v1) < 0.0
    b2 = sign((px, py), v1, v2) < 0.0
    b3 = sign((px, py), v2, v0) < 0.0

    return ((b1 == b2) and (b2 == b3))

def get_z_value(px, py, v0, v1, v2):
  
    def edge_function(v0, v1, p):
        return (p[0] - v0[0]) * (v1[1] - v0[1]) - (p[1] - v0[1]) * (v1[0] - v0[0])

    area = edge_function(v0, v1, v2)
    w0 = edge_function(v1, v2, (px, py)) / area
    w1 = edge_function(v2, v0, (px, py)) / area
    w2 = edge_function(v0, v1, (px, py)) / area

    return w0 * v0[2] + w1 * v1[2] + w2 * v2[2]


image = Image.new("RGB",(1200,1200), (255,255,255))
points=(loadMesh('african_head.obj'))[0]
texture=(loadMesh('african_head.obj'))[1]
width, height = 1400, 1400
image = z_buffer(points,texture, norm,width, height)
image.show()
